In [27]:
using Pkg;

In [ ]:
Pkg.add("DataFrames");
Pkg.add("JSON");
Pkg.add("CSV");
Pkg.add("NamedTupleTools");
Pkg.add("Lazy");

In [3]:
using DataFrames;
import JSON;
using Lazy: @>, @>>, @as

In [1]:
# voteResultsHistory.json is exported in the software under DAO governance section
dao_proposals_file = "voteResultsHistory.json"
json = open(JSON.parse, dao_proposals_file, write=false);

LoadError: UndefVarError: JSON not defined

In [32]:
a_cycle = @> json first;
a_proposal = @> a_cycle get("proposals", missing) first;
a_vote = @> a_proposal get("votes", missing) first;

In [ ]:
a_cycle

In [ ]:
a_proposal

In [ ]:
a_vote

In [5]:
delete_keys!(d, keys) = begin
    for k in keys delete!(d, k) end
    return d
end
parse_proposal(proposal, cycleIndex) = begin
    # drop some keys to conserve memory
    delete_keys!(proposal, ["votes","proposal.txId","proposal.role.uid"])
    # persist proposal's cycle
    proposal["cycleIndex"] = cycleIndex
    return proposal
end;

In [6]:
function parse_cycle(cycle)
    index = cycle["cycleIndex"]
    proposals = map(p -> parse_proposal(p, index), cycle["proposals"])
    return (index = index, startTime = cycle["startTime"], proposals = proposals)
end;

In [7]:
cycles = map(parse_cycle, json);

In [8]:
cycles_df = @>> cycles map(c -> (index=c.index, startTime=c.startTime)) DataFrame;

In [9]:
proposals = begin
    proposals_grouped = map(c -> c.proposals, cycles)
    vcat(proposals_grouped...)
end;

In [10]:
using NamedTupleTools: namedtuple;
string_dict_to_namedtuple(d) = namedtuple(Symbol.(keys(d)), values(d));

In [11]:
missing_keys = @>> proposals map(keys) reduce(union);

In [12]:
add_key_if_missing(d, k) = get!(d, k, missing)
add_missing_keys(p, missing_keys) = begin
    @>> missing_keys foreach(k -> add_key_if_missing(p,k))
    p
end

df = @>> begin
    proposals
    map(p -> add_missing_keys(p, missing_keys))
    map(string_dict_to_namedtuple)
    DataFrame
end;

In [13]:
get_rid_of_dots(s) = replace(s, "." => "_")
get_rid_of_symbol_dots(sy) = @> sy string get_rid_of_dots
rename!(get_rid_of_symbol_dots, df);

In [14]:
import CSV;

proposals_output_path = "bisq-proposals.csv"
CSV.write(proposals_output_path, df)

cycles_output_path = "bisq-cycles.csv"
CSV.write(cycles_output_path, cycles_df);